# Dalle ANN alle reti deep

Prima di passare alle Deep Neural Networks (DNN) è importante analizzare bene alcuni aspetti delle ANN che ne hanno limitato la diffusione e l'uso fino al recente avvento del deep learnig.

Tra le varie cose, le reti deep prendono il loro nome proprio dal fatto che sono molto profonde, ossia composte da una sequenza di numerosi livelli. Dunque, uno degli aspetti cruciali da investigare è l'effetto che la profondità della rete ha sulle performance di un MLP. A tal fine, partiamo proprio dall'esempio dell'esercizio della lezione precedente e mostriamo come varia la curva dell'accuracy al variare del numero livelli, ma tenendo costante il numero complessivo di neuroni.

L'esecuzione della fase di addestramento del codice seguente potrebbe richiedere **una quantità di tempo considerevole**. Per questo motivo, è presente una cella alternativa in grado di caricare i parametri e le variabili risultanti da un'esecuzione della cella precedente.

In [ ]:
#####################################################################################
#                                                                                   #
#   ATTENZIONE: eseguire questa cella SOLO SE SI VUOLE EFFETTUARE L'ADDESTRAMENTO.  #
#               Eseguire la cella seguente se si vogliono caricare le variabili     #
#               necessarie già pronte per l'uso. L'esecuzione di questa cella può   #
#               richiedere diversi minuti!                                          #
#                                                                                   #
##################################################################################### 

import numpy as np
import matplotlib.pyplot as plt
import sklearn 
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.neural_network import MLPClassifier

#Caricamento dei dati
digits = load_digits()

#Preparazione del dataset
dati = digits.data           
classi = digits.target

dataTrain, dataTest, classiTrain, classiTest = train_test_split(dati, classi, test_size = 0.20)

scaler = StandardScaler()
scaler.fit(dataTrain)

dataTrainScalato = scaler.transform(dataTrain)
dataTestScalato = scaler.transform(dataTest)

#Range dei valori per le epoche
epochRange = np.arange(1, 500, 5)

#Addestramento per due MLP, uno con 3 livelli hidden e uno con 9
trainScores_MLP3, validationScores_MLP3 = validation_curve(MLPClassifier(hidden_layer_sizes=(10,50,10), solver='sgd', activation='tanh'), 
                                             dataTrainScalato, 
                                             classiTrain, 
                                             param_name="max_iter", 
                                             param_range=epochRange,
                                             cv=3, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

trainScores_MLP9, validationScores_MLP9 = validation_curve(MLPClassifier(hidden_layer_sizes=(10,10,10,10,10,5,5,5,5), solver='sgd', activation='tanh'), 
                                             dataTrainScalato, 
                                             classiTrain, 
                                             param_name="max_iter", 
                                             param_range=epochRange,
                                             cv=3, 
                                             scoring="accuracy", 
                                             n_jobs=-1)

In [ ]:
#####################################################################################
#                                                                                   #
#   ATTENZIONE: eseguire questa cella SOLO SE SI VOGLIONO CARICARE I DATI PRONTI.   #
#               Non è necessario eseguire questa cella se si è eseguita quella      #
#               precedente, dove è stato effettuato l'addestramento.                #
#                                                                                   #
##################################################################################### 

import numpy as np
import matplotlib.pyplot as plt
import pickle

with open('preElaborati1.pkl', 'rb') as f:
    trainScores_MLP3, validationScores_MLP3, trainScores_MLP9, validationScores_MLP9, epochRange = pickle.load(f)

print('Dati pre-elaborati caricati con successo!')

In [ ]:
#Visualizzazione
trainMean_MLP3 = np.mean(trainScores_MLP3, axis=1)
trainStd_MLP3 = np.std(trainScores_MLP3, axis=1)
valMean_MLP3 = np.mean(validationScores_MLP3, axis=1)
valStd_MLP3 = np.std(validationScores_MLP3, axis=1)

trainMean_MLP9 = np.mean(trainScores_MLP9, axis=1)
trainStd_MLP9 = np.std(trainScores_MLP9, axis=1)
valMean_MLP9 = np.mean(validationScores_MLP9, axis=1)
valStd_MLP9 = np.std(validationScores_MLP9, axis=1)

plt.figure()

plt.subplot(1, 2, 1) 
plt.plot(epochRange, trainMean_MLP3, label="Training Accuracy", color="black")
plt.plot(epochRange, valMean_MLP3, label="Cross-validation Accuracy", color="dimgrey")
plt.fill_between(epochRange, trainMean_MLP3 - trainStd_MLP3, trainMean_MLP3 + trainStd_MLP3, color="gray")
plt.fill_between(epochRange, valMean_MLP3 - valStd_MLP3, valMean_MLP3 + valStd_MLP3, color="gainsboro")
plt.title("Tre livelli")
plt.xlabel("Epoche")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.legend(loc="best")

plt.subplot(1, 2, 2) 
plt.plot(epochRange, trainMean_MLP9, label="Training Accuracy", color="black")
plt.plot(epochRange, valMean_MLP9, label="Cross-validation Accuracy", color="dimgrey")
plt.fill_between(epochRange, trainMean_MLP9 - trainStd_MLP9, trainMean_MLP9 + trainStd_MLP9, color="gray")
plt.fill_between(epochRange, valMean_MLP9 - valStd_MLP9, valMean_MLP9 + valStd_MLP9, color="gainsboro")
plt.title("Nove livelli")
plt.xlabel("Epoche")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.legend(loc="best")

Come si vede dai grafici delle curve di addestramento, sebbene in entrambi i casi il numero totale di neuroni nei livelli hidden sia sempre 70, la presenza di un maggior numero di livelli ha un grosso impatto sulla capacità di apprendimento della rete. Sebbene possa risultare inatteso, questo comportamento è (principalmente) legato alla propagazione dei gradienti durante la fase di addestramento. Difatti, la presenza di un numero maggiore di livelli può causare due fenomeni noti come Vanishing Gradient e Exploding Gradient: 

*   nel primo caso, il valore dei gradienti (usato per aggiornare i pesi dei neuroni) inizia a diventare cosi piccolo da risultare sostanzialmente insufficiente a generare una variazione efficace dei pesi (e quindi la rete non apprende)
*   nel secondo caso, il gradiente si accumula, causando grosse variazioni dei pesi e dunque grosse oscillazioni nei pesi (con conseguente oscillazione delle metriche di performance)

Tale problema ha limitato lo sviluppo in profondità delle ANN che, di conseguenza, hanno invece visto lo sviluppo di architetture con pochi, ma ampi, livelli.

Sebbene questo problema sussista tutt'ora (e diversi esperti credano possa essere solo limitato e mai completamente risolto), tre fattori hanno contribuito alla nascita delle deep neural networks, infrangendo il limite dei pochi livelli hidden:

1.   Lo sviluppo di nuove soluzioni hardware (più performanti e capaci di lavorare con rappresentazioni su più bit)
2.   La disponibilità di elevate moli di dati (necessari al crescere del numero complessivo di neuroni da addestrare)
3.   Il design di nuove funzioni di attivazione e algoritmi di ottimizzazione, progettati per risentire meno del problema

Scikit-learn supporta tutte queste caratteristiche, dato che è in grado di sfruttare hardware recenti (come ad esempio l'accellerazione hardware basata su GPU), e permette di utilizzare diversi ottimizzatori e funzioni di attivazione. 

Ciononostante, scikit-learn non è lo strumento più adatto per il design e l'addestramento di Deep Neural Networks. Infatti, scikit-learn è progettato per supportare in maniera uniforme e coerente una vasta gamma di algoritmi e tecniche di machine learning, le cui caratteristiche e necessità sono tuttavia differenti da quelle proprie del mondo delle deep neural network. 





In [ ]:
#TASK: che succede se si ripete l'esecuzione utilizzando un altro ottimizzatore e/o un'altra funzione di attivazione?


# ANN con PyTorch

PyTorch (anche in questo caso "Py" si pronuncia come $\pi$ in inglese) è un framework open source nato per il design e l'addestramento di deep neural networks. Basato su Torch, ha due caratteristiche che lo rendono particolarmente efficace per il Deep Learning: 

*   L'introduzione di tensori (molto simili agli NDarray) nativamente in grado di essere manipolati da GPU
*   Differenziazione automatica (ossia la capacità di determinare la derivata di funzioni a piacere ottenute come composizione di funzioni base messe a disposizione dal framework)

Per capire i vantaggi di questi punti, riprendiamo lo stesso problema della classificazione delle cifre e progettiamolo nuovamente usando PyTorch.

Partiamo dagli import



In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader

print(torch.__version__)
print(torchvision.__version__)

Escludendo NumPy e Matplotlib, ci sono diverse a cui prestare attenzione

*   `torch` è namespace di PyTorch
*   `torchvision` è il package che contiene numerosi modelli, dataset e algoritmi per la computer vision. Nel nostro caso importiamo due moduli: `dataset`, che contiene una serie di dataset pronti per l'uso; `transforms`, che fornisce alcune primitive per la manipolazione dei dati
*   `torch.nn` è il modulo di alto livello (basato sulle sottostanti funzionalità di derivazione automatica) progettato per definire complesse (o meno) funzioni di attivazione e di loss per reti neurali, le cui derivate saranno automaticamente calcolate
*   `torch.optim` è il modulo che include differenti algoritmi di ottimizzazione 
*   `torch.Tensor` introduce il concetto di tensore, ossia un oggetto estremamente simile agli NDarray di NumPy, ma sul quale possono nativamente operare alcune GPU NVidia
*   `torch.utils.data.DataLoader` è il modulo che permette di caricare, in maniera rapida ed efficiente, i dati utilizzati poi dalla rete 

Si noti che l'import del modulo `Tensor` non è strettamente necessario, ed è stato qui riportato principalmente per questioni didattiche.

Come sempre, dettagli e funzionalità supportate possono variare tra le varie versioni. In questo corso fare riferimento alla versione `1.5.0` per PyTorch e `0.6.0` per Torchvision

## Caricamento e preparazione dei dati

Come per scikit-learn, è possibile utilizzare alcune funzionalità di PyTorch per accedere ad alcuni dataset famosi. Mnist è tra questi. Per scaricarlo, usiamo il modulo `torchvision.datasets`.

Come fatto nel caso di scikit-learn, anche in questo caso useremo la versione linearizzata dell'immagine come vettore di feature per la nostra ANN. In questo caso usiamo le funzionalità messe a disposizione da `torchvision.transforms`, ricordandoci che in PyTorch tutto è un tensore

In [ ]:
#Definizione della funzione di linearizzazione e normalizzazione dell'input
trasf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainingSet = datasets.MNIST('MNIST_data/', download=True, train=True, transform=trasf)
testSet = datasets.MNIST('MNIST_data/', download=True, train=False, transform=trasf)

trainingLoader = DataLoader(trainingSet, batch_size = 64, shuffle = True)
testLoader = DataLoader(testSet, batch_size = 64, shuffle = True)

Queste poche righe di codice ci danno già la possibilità di approfondire diversi aspetti

Come fatto nel caso di scikit-learn, non solo dobbiamo linearizzare l'immagine, ma dobbiamo anche normalizzare i valori dei pixel. Data la necessità di effettuare due trasformazioni, usiamo il metodo `transforms.Compose` per definire un array di trasformazioni da applicare ai dati di input

1.   `ToTensor()` permette di trasformare i dati da `ndarry` a `torch.Tensor`;
2.   il metodo `Normalize` accetta un valore di media e uno di varianza per ognuno dei canali (es. RGB per le immagini naturali) per poi eseguire l'operazione di normalizzazione. MNIST contiene immagini in bianco e nero, su un singolo canale (dunque è sufficiente avere un singolo valore di media ed un singolo valore di varianza). Si noti che in linea teorica, cosi come fatto per scikit-learn, avremmo dovuto calcolare la media e la varianza sul training set e poi applicare la trasformazione al training e al test set. Nel caso di MNIST, tuttavia, data la particolare tipologia di immagini, usare il valore 0.5 per entrambi permette di ottenere (con buona approssimazione) la trasformazione desiderata, senza dover "sprecare" potenza computazionale per il calcolo dei valori precisi.

Un altro metodo interessante è `datasets.MNIST`. In questo caso il metodo scaricherà le immagini nella cartella desiderata, prelevandole dal training set o dal test set in base al valore del parametro booleano `train`. Si noti anche che il metodo accetta in ingresso la funzione di trasformazione precedentemente definita e discussa. La motivazione sta nel fatto che il dataset cosi definito è poco più di un funzionale il cui compito è memorizzare tutte le informazioni che saranno poi utilizzate dal modulo di caricamento vero e proprio. 

Nel caso specifico, chi si occupa del caricamento dei dati è il `DataLoader`. Si tratta di un modulo che permette di disaccoppiare il caricamento dei dati dalla loro elaborazione. Questo passaggio si rende molto utile in quanto l'addestramento delle reti deep tende (praticamente sempre) ad essere più efficace se eseguito per batch di dati. In sostanza, invece di aggiornare i pesi dei neuroni per ogni campione analizzato (strategia nota come "training by pattern") o dopo aver elaborato tutti i campioni del training set (strategia nota come "training by epoch"), si aggiornano i pesi ad ogni batch, ossia dopo aver elaborato una porzione dei dati di training (noto come "batch training"). La dimensione del batch è impostabile tramite il parametro batch_size (64 nell'esempio). In questo contesto l'uso di un `DataLoader` si rivela estremamente utile in quanto sarà quest'ultimo ad occuparsi di generare una opportuna suddivisione in batch, passandoli di volta in volta all'algoritmo di addestramento vero e proprio.

## Definizione della rete e addestramento

A differenza di quanto visto con scikit-learn, in PyTorch è compito del programmatore definire i dettagli dell'architettura e del processo di addestramento. Sebbene all'inizio questo possa risultare complesso, con il tempo la libertà espressiva permessa da questo approccio ripagano ampiamente lo sforzo iniziale. PyTorch fornisce diverse primitive per supportare tale processo.

## Design della rete

Il primo passo è la definizione dell'architettura della nostra rete neurale artificiale

In [ ]:
#Caratteristiche dei liveli
dimensioneInput = 784         #Dato che in questo caso le immagini in ingresso sono 28x28x1 pixel
livelliHidden = [128, 64]     #Numero di neuroni per ogni livello hidden
dimensioneOutput = 10         #Numero di neuroni nel livello di output (pari al numero di classi)

#Definizione della rete
modello = nn.Sequential(nn.Linear(dimensioneInput, livelliHidden[0]),
                      nn.ReLU(),
                      nn.Linear(livelliHidden[0], livelliHidden[1]),
                      nn.ReLU(),
                      nn.Linear(livelliHidden[1], dimensioneOutput),
                      nn.LogSoftmax(dim=1))

#Visualizzazione
print(modello)

Il metodo `nn.Sequential` permette la definizione di una ANN di tipo feedforward, costuita collegando in sequenza i livelli neurali che sono passati come input. Nel caso specifico costruiamo una rete con tre livelli di tipo `nn.Linear`, definendo per ognuno di essi il numero di ingressi attesi e uscite da generare. Per i primi due livelli, la funzione di attivazione usata è una *ReLu* (computazionalmente più efficiente). Per l'ultimo livello si usa invece una *LogSoftmax* (particolarmente adatta a gestire la classificazione multi classe, oltre ad essere caratterizzata da una elevata stabilità numerica e una forte penalizzazione per gli errori commessi durante l'addestramento).

## Scelta del device di addestramento

Definita l'architettura del modello, si può caricare il modello stesso sulla GPU (se disponibile) per velocizzare i tempi di addestramento. PyTorch permette di verificare velocemente se il sistema utilizzato presenta una GPU compatibile. In caso contrario, PyTorch userà la CPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
modello.to(device)

## Definizione dell'algoritmo di addestramento

Prima di procedere con la definizione dell'algoritmo di apprendimento è necessario definire la *funzione di errore* (in inglese nota come loss function) che sarà minimizzata dall'algoritmo durante il processo di apprendimento. In questo esempio useremo la "Negative Log-Likelihood" (implementata dal modulo `nn.NLLLoss()`) in quanto ben si presta ad una classificazione multi-classe. 

In [ ]:
lossFun = nn.NLLLoss()     
optimizer = optim.SGD(modello.parameters(), lr = 0.003, momentum = 0.9)     #lr = Learning rate            

Dopo aver definito la funzione di loss e l'algoritmo di ottimizzazione che si desidera utilizzare, possiamo realizzare l'algoritmo di addestramento vero e proprio. Si tratta sostanzialmente di realizzare un ciclo che iteri l'addestramento per ognuno dei batch generati dal Dataloader. Questo processo, deve poi essere ripetuto tante volte quante vogliamo che siano il numero di epoche di addestramento. 

L'esecuzione della fase di addestramento del codice seguente potrebbe richiedere **una quantità di tempo considerevole**. Per questo motivo, è presente una cella alternativa in grado di caricare i parametri e le variabili risultanti da un'esecuzione della cella precedente.

In [ ]:
#####################################################################################
#                                                                                   #
#   ATTENZIONE: eseguire questa cella SOLO SE SI VUOLE EFFETTUARE L'ADDESTRAMENTO.  #
#               Eseguire la cella seguente se si vogliono caricare le variabili     #
#               necessarie già pronte per l'uso. L'esecuzione di questa cella può   #
#               richiedere diversi minuti!                                          #
#                                                                                   #
##################################################################################### 

epoche = 15
lossTraining = []

#Ciclo di addestramento sulle epoche
for e in range(epoche):
    #Variabile di appoggio per la loss dell'epoca corrente
    lossEpoca = 0

    #Ciclo sui batch, per l'e-esima epoca
    for datiBatch, labelsBatch in trainingLoader:

        #Linearizzazione delle immagini nel batch corrente
        datiBatch = datiBatch.view(datiBatch.shape[0], -1)

        #Gestione dell'eventuale GPU
        if torch.cuda.is_available():
          datiBatch = datiBatch.cuda()
          labelsBatch = labelsBatch.cuda()
            
        #Passo di forward
        output = modello(datiBatch) 
        loss = lossFun(output, labelsBatch) 

        #Azzeramento dei gradienti
        optimizer.zero_grad()
       
        #Passo di back-propagation dell'errore
        loss.backward()
        
        #Aggiornamento dei pesi del modello
        optimizer.step()
        
        #Aggiornamento della loss per l'epoca corrente, sommando la loss del batch attuale
        lossEpoca += loss.item()*datiBatch.size(0)    

    #Aggiornamento dell'array delle loss (per ogni epoca)
    lossTraining.append(lossEpoca/len(trainingLoader))
    
    #Stampa delle informazioni correnti
    print("Epoca {} - Loss sul TrainingSet: {}".format(e, lossTraining[-1:]))

In [ ]:
#####################################################################################
#                                                                                   #
#   ATTENZIONE: eseguire questa cella SOLO SE SI VOGLIONO CARICARE I DATI PRONTI.   #
#               Non è necessario eseguire questa cella se si è eseguita quella      #
#               precedente, dove è stato effettuato l'addestramento. Nota che il    #
#               salvataggio e caricamento sono qui riportati solo per questioni     #
#               legate al tempo di esecuzione dell'algoritmo. Nelle prossime        #
#               lezioni sarà spiegata in dettaglio la procedura di salvataggio e    #
#               caricamento di ANN/CNN con PyTorch.                                 #
#                                                                                   #
##################################################################################### 

import pickle

with open('preElaborati2.pkl', 'rb') as f:
    epoche, lossTraining = pickle.load(f)
modello.load_state_dict(torch.load('preElaborato.pth', map_location=torch.device('cpu')))

print('Dati pre-elaborati caricati con successo!')

Quello che abbiamo fatto è richiedere, per ogni epoca un batch (dati e labels) al Dataloader definito precedentemente. In particolare, per i dati, abbiamo inoltre effettuato la linerizzazione (dato che consideriamo i valori dei pixel come features). Nota che anche in questo caso abbiamo dovuto controllare se fosse disponibile una GPU compatibile. In quest'ultimo caso è necessario infatti comunicare esplicitamente al sistema che il modello (e quindi eventuali dati di cui ha bisogno e/o che genera) vanno gestiti tramitte GPU.

Si noti che prima di ogni passo di backward, resettiamo i gradienti. Questo passaggio è necessario in quanto PyTorch accumula i gradienti per esecuzioni successive di back-propagation. Questa funzionalità, molto utile nel caso dell'addestramento di reti ricorrenti (che non vedremo in questo corso), è invece deleterio nel caso dell'addestramento di reti di tipo feed-forward.

Due variabili meritano un ulteriore appprofondimento: `lossEpoca` e `lossTraining`. Per la prima nota che moltiplichiamo il valore di loss calcolato per il numero di elementi nel batch corrente. Questo passaggio è necessario in quanto quella che si ottiene dall'applicazione di `criterion` è la loss media del batch attuale (dunque moltiplicandola per il totale degli elementi in quel batch, otteniamo la loss complessiva del batch). Viceversa, dato che `lossEpoca` a valle del passo precedente ha accumulato le loss complessive per tutti gli elementi di tutti i batch (ossia tutto il training set), volendo visualizzare la loss media dell'intera epoca, dobbiamo dividere tale valore per il numero di batch in cui il training set è stato diviso.

A questo punto, possiamo visualizzare graficamente l'andamento della loss

In [ ]:
plt.plot(np.arange(0, epoche, 1), lossTraining, label="Training Loss", color="black")
plt.xlabel("Epoche")
plt.ylabel("Loss")
plt.tight_layout()
plt.legend(loc="best")

Si noti che la prima epoca è etichettata come 0 solo per questioni di coerenza con l'indexing di Python. Tuttavia, il primo valore di loss si è in realtà ottenuto alla fine della prima epoca.

## Valutazione delle performance

L'ultimo passaggio che resta da compiere è la valutazione delle performance sul test set

In [ ]:
#####################################################################################
#                                                                                   #
#   L'esecuzione di questa cella può richiedere diversi minuti.                     #
#                                                                                   #
#####################################################################################

campioniCorretti, campioniTotali = 0, 0                      #Valori di accumulo delle immagini analizzate
for datiTest, lablesTest in testLoader:
  for i in range(len(lablesTest)):
    campione = datiTest[i].view(1, 784)

    #Gestione GPU
    if torch.cuda.is_available():
      campione = campione.cuda()
    
    #Stiamo in inferenza, quindi non serve calcolare i gradienti
    with torch.no_grad():
      output = modello(campione)

    #Calcolo della classe predetta
    probs = torch.exp(output)
    probsList = list(probs.cpu().numpy()[0])
    classePred = probsList.index(max(probsList))

    #Valutazione della correttezza della predizione
    classeVera = lablesTest.numpy()[i]
    if(classeVera == classePred):
      campioniCorretti += 1
    campioniTotali += 1

print("Numero di campioni nel test set =", campioniTotali)
print("\nAccuracy sul test set =", (campioniCorretti/campioniTotali))

Si noti che per ottenere le probabilità predette abbiamo valutato l'esponenziale dell'output (dato che avevamo scelto di usare la LogSoftmax). Tuttavia questo passaggio è meramente didattico, in quanto il logaritmo preserva l'ordinamento dei valori (e quindi tutti i passaggi successivi avrebbero comunque funzionato correttamente)

# Esercizio

Ripetere l'esercizio sull'addestramento della ANN per la classificazione del dataset MNIST, includendo 

*   Divisione del training set in training e validation
*   Aggiunta della visualizzazione dell'accuracy e della loss per il training set e per validation set alla fine di ogni epoca
*   Visualizzazione delle training curve
*   Calcolo dell'errore sul test set

Per lo split del dataset è possibile usare la seguente funzione

```
def train_val_dataset(dataset, validation=0.20):
    trainIdx, valIdx = train_test_split(list(range(len(dataset))), test_size=validation)
    trainingSet = Subset(dataset, trainIdx)
    validationSet = Subset(dataset, valIdx)
    return(trainingSet, validationSet)
```



# Soluzione


In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch import nn, optim, Tensor
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

#Funzione per lo split del training set
def train_val_dataset(dataset, validation=0.20):
    trainIdx, valIdx = train_test_split(list(range(len(dataset))), test_size=validation)
    trainingSet = Subset(dataset, trainIdx)
    validationSet = Subset(dataset, valIdx)
    return(trainingSet, validationSet)

#Creiamo un metodo che, a partire da Dati e Labels, calcoli Loss e Accuracy (dato che lo useremo per training, test e validation)
def valutaOutputEPerformance(modello, criterion, dati, labels):
  if torch.cuda.is_available():
    dati = dati.cuda()
    labels = labels.cuda()

  output = modello(dati) 
  loss = criterion(output, labels) 

  campioniCorretti, campioniTotali = 0, 0  
  for i in range(len(labels)):
    campione = dati[i].view(1, 784)
    
    with torch.no_grad():    
      outputCampione = modello(campione)

    #Calcolo della classe predetta
    probs = torch.exp(outputCampione)
    probsList = list(probs.cpu().numpy()[0])
    classePred = probsList.index(max(probsList))

    #Valutazione della correttezza della predizione
    classeVera = labels.cpu().numpy()[i]
    if(classeVera == classePred):
      campioniCorretti += 1
    campioniTotali += 1

  accuracy = campioniCorretti/campioniTotali
  return(output, loss, accuracy)

trasf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainingSet = datasets.MNIST('MNIST_data/', download=True, train=True, transform=trasf)
trainingSet, validationSet = train_val_dataset(trainingSet, validation=0.20)
testSet = datasets.MNIST('MNIST_data/', download=True, train=False, transform=trasf)
trainingLoader = DataLoader(trainingSet, batch_size=64, shuffle=True)
validationLoader = DataLoader(validationSet, batch_size=64, shuffle=True)
testLoader = DataLoader(testSet, batch_size=64, shuffle=True)

dimensioneInput = 784         
livelliHidden = [128, 64]     
dimensioneOutput = 10         

modello = nn.Sequential(nn.Linear(dimensioneInput, livelliHidden[0]),
                      nn.ReLU(),
                      nn.Linear(livelliHidden[0], livelliHidden[1]),
                      nn.ReLU(),
                      nn.Linear(livelliHidden[1], dimensioneOutput),
                      nn.LogSoftmax(dim=1))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
modello.to(device)

criterion = nn.NLLLoss()                      
optimizer = optim.SGD(modello.parameters(), lr=0.003, momentum=0.9)
epoche = 15 
lossTraining = []
accuracyTraining = []
lossValidation = []
accuracyValidation = []

for e in range(epoche):
  lossTrainEpoca = 0
  lossValEpoca = 0
  for datiBatchTrain, labelsBatchTrain in trainingLoader:
      datiBatchTrain = datiBatchTrain.view(datiBatchTrain.shape[0], -1)   
      outputTrain, lossTrain, accuracyTrain = valutaOutputEPerformance(modello, criterion, datiBatchTrain, labelsBatchTrain)      
      optimizer.zero_grad()
      lossTrain.backward()
      optimizer.step()
      lossTrainEpoca += lossTrain.item()*datiBatchTrain.size(0)    
  lossTraining.append(lossTrainEpoca/len(trainingLoader))
  accuracyTraining.append(accuracyTrain)

  #Calcolo delle performance sul validation set, ad ogni fine epoca
  for datiBatchVal, labelsBatchVal in validationLoader:
      datiBatchVal = datiBatchVal.view(datiBatchVal.shape[0], -1)   
      outputVal, lossVal, accuracyVal = valutaOutputEPerformance(modello, criterion, datiBatchVal, labelsBatchVal)      
      lossValEpoca += lossVal.item()*datiBatchVal.size(0)    
  lossValidation.append(lossValEpoca/len(validationLoader))
  accuracyValidation.append(accuracyVal)
  
  print("Epoca {}:".format(e))
  print("Loss sul Training Set: {} - Accuracy sul Training Set: {}".format(lossTraining[-1:], np.mean(accuracyTraining)))
  print("Loss sul Validation Set: {} - Accuracy sul Validation Set: {}".format(lossValidation[-1:], np.mean(accuracyValidation)))
  print("--------------------------------------------------------------------------------------------------")

#Visualizzazione delle training curves
plt.figure()

plt.subplot(1, 2, 1) 
plt.plot(np.arange(0, epoche, 1), lossTraining, label="Training Loss", color="black")
plt.plot(np.arange(0, epoche, 1), lossValidation, label="Validation Loss", color="dimgrey")
plt.xlabel("Epoche")
plt.ylabel("Loss")
plt.tight_layout()
plt.legend(loc="best")

plt.subplot(1, 2, 2) 
plt.plot(np.arange(0, epoche, 1), accuracyTraining, label="Training Accuracy", color="black")
plt.plot(np.arange(0, epoche, 1), accuracyValidation, label="Validation Accuracy", color="dimgrey")
plt.xlabel("Epoche")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.legend(loc="best")

#Calcolo delle performance finali sul test set
accuracyTest = []
campioniTotaliTest = 0
for datiBatchTest, labelsBatchTest in testLoader:
  datiBatchTest = datiBatchTest.view(datiBatchTest.shape[0], -1)   
  outputTest, lossTest, accuracyT = valutaOutputEPerformance(modello, criterion, datiBatchTest, labelsBatchTest)       
  accuracyTest.append(accuracyT)
  campioniTotaliTest += datiBatchTest.size(0);
print("Numero di campioni nel test set =", campioniTotaliTest)
print("\nAccuracy sul test set =", np.mean(accuracyTest))